Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

# Repro for [NVIDIA/spark-rapids#8625](https://github.com/NVIDIA/spark-rapids/issues/8625)

In [1]:
import datetime
import findspark
import os

In [2]:
rapids_version = '23.08.0-SNAPSHOT'
spark_version = '3.4.1'

In [3]:
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ'] = 'UTC'

home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
spark_home = f"{home_dir}/dist/spark-3.4.1-bin-hadoop3"
cores_per_exec = 1
spark_master = f"local[{cores_per_exec}]"

In [4]:
findspark.init(spark_home = spark_home)
findspark.add_jars(f"{home_dir}/gits/NVIDIA/spark-rapids/dist/target/rapids-4-spark_2.12-{rapids_version}-cuda11.jar")
import pyspark
spark = pyspark.sql.SparkSession.builder \
    .appName('RAPIDS PySpark Notebook') \
    .master(spark_master) \
    .config(map = {
        'spark.plugins': 'com.nvidia.spark.SQLPlugin',
        'spark.rapids.sql.explain': 'ALL', 
        'spark.sql.adaptive.enabled': False,
    }) \
    .getOrCreate()

23/07/14 14:48:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/14 14:48:25 WARN RapidsPluginUtils: RAPIDS Accelerator 23.08.0-SNAPSHOT using cudf 23.08.0-SNAPSHOT.
23/07/14 14:48:25 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
23/07/14 14:48:25 WARN RapidsPluginUtils: spark.rapids.sql.explain is set to `ALL`. Set it to 'NONE' to suppress the diagnostics logging about the query placement on the GPU.


In [5]:
ts = datetime.datetime(5000, 7, 14, 7, 38, 45, 418688)
ts

datetime.datetime(5000, 7, 14, 7, 38, 45, 418688)

In [6]:
spark.createDataFrame([(ts,),], 'ts_ntz timestamp').createOrReplaceTempView('df1')
spark.sql("SELECT * FROM df1").show(truncate = False)
spark.sql("SELECT * FROM df1").printSchema()

23/07/14 14:48:31 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
  *Exec <ProjectExec> will run on GPU
    *Expression <Alias> cast(ts_ntz#0 as string) AS ts_ntz#4 will run on GPU
      *Expression <Cast> cast(ts_ntz#0 as string) will run on GPU
    ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
      @Expression <AttributeReference> ts_ntz#0 could run on GPU



+--------------------------+
|ts_ntz                    |
+--------------------------+
|5000-07-14 07:38:45.418688|
+--------------------------+

root
 |-- ts_ntz: timestamp (nullable = true)



In [7]:
spark.conf.get('spark.sql.parquet.outputTimestampType')

'INT96'

In [8]:
spark.sql("SELECT MAX(*) FROM df1").write.mode('overwrite').parquet('/tmp/int96_out.parquet')

23/07/14 14:48:32 WARN GpuOverrides: 
*Exec <DataWritingCommandExec> will run on GPU
  *Output <InsertIntoHadoopFsRelationCommand> will run on GPU
  *Exec <WriteFilesExec> will run on GPU
    *Exec <HashAggregateExec> will run on GPU
      *Expression <AggregateExpression> max(ts_ntz#0) will run on GPU
        *Expression <Max> max(ts_ntz#0) will run on GPU
      *Expression <Alias> max(ts_ntz#0)#9 AS max(ts_ntz)#10 will run on GPU
      *Exec <ShuffleExchangeExec> will run on GPU
        *Partitioning <SinglePartition$> will run on GPU
        *Exec <HashAggregateExec> will run on GPU
          *Expression <AggregateExpression> partial_max(ts_ntz#0) will run on GPU
            *Expression <Max> max(ts_ntz#0) will run on GPU
          ! <RDDScanExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.RDDScanExec
            @Expression <AttributeReference> ts_ntz#0 could run on GPU

23/07/14 14:48:33 ERROR Utils: Aborting task
jav

Py4JJavaError: An error occurred while calling o58.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.rapids.GpuFileFormatWriter$.writeAndCommit(GpuFileFormatWriter.scala:314)
	at org.apache.spark.sql.rapids.GpuFileFormatWriter$.executeWrite(GpuFileFormatWriter.scala:329)
	at org.apache.spark.sql.rapids.GpuFileFormatWriter$.write(GpuFileFormatWriter.scala:199)
	at org.apache.spark.sql.rapids.GpuInsertIntoHadoopFsRelationCommand.runColumnar(GpuInsertIntoHadoopFsRelationCommand.scala:184)
	at com.nvidia.spark.rapids.GpuDataWritingCommandExec.sideEffectResult$lzycompute(GpuDataWritingCommandExec.scala:117)
	at com.nvidia.spark.rapids.GpuDataWritingCommandExec.sideEffectResult(GpuDataWritingCommandExec.scala:116)
	at com.nvidia.spark.rapids.GpuDataWritingCommandExec.internalDoExecuteColumnar(GpuDataWritingCommandExec.scala:140)
	at com.nvidia.spark.rapids.GpuExec.doExecuteColumnar(GpuExec.scala:349)
	at com.nvidia.spark.rapids.GpuExec.doExecuteColumnar$(GpuEx